# Using Machine Learning Models to Predict ESG Rating

In [3]:
# import packages
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import KNNImputer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
import xgboost as xgb

## Data preprocessing

In [4]:
# load the merged Excel file after obtaining data from SDG2000, MSCI, Refinitiv, and Yahoo Finance
df = pd.read_excel('ESG_final_v3.xlsx')

In [5]:
# check the dataset
df.head()

,Name,Industry,Headquarters,Region,Food and Agriculture Transformation,Decarbonisation and Energy Transformation,Nature and Biodiversity Transformation,Digital Transformation,Financial Transformation,Urban Transformation,...,esgPerformance,coal,pesticides,adult,percentile,smallArms,environmentScore,governancePercentile,militaryContract,ESG_rating
0,3M,Chemicals,United States of America,North America,NaN,NaN,Yes,NaN,NaN,NaN,...,OUT_PERF,0.0,0.0,0.0,73.53,0.0,12.33,NaN,0.0,92.0
1,A.P. Moller - Maersk,Logistics,Denmark,Europe & Central Asia,NaN,Yes,Yes,NaN,NaN,NaN,...,AVG_PERF,0.0,0.0,0.0,33.43,0.0,9.68,NaN,0.0,92.0
2,ABB,Capital Goods,Switzerland,Europe & Central Asia,NaN,Yes,NaN,NaN,NaN,NaN,...,UNDER_PERF,0.0,0.0,0.0,16.50,0.0,4.01,NaN,0.0,94.0
3,AbbVie,Pharmaceuticals & Biotechnology,United States of America,North America,NaN,NaN,Yes,NaN,NaN,NaN,...,AVG_PERF,0.0,0.0,0.0,54.41,0.0,1.12,NaN,0.0,81.0
4,Abercrombie & Fitch,Apparel & Footwear,United States of America,North America,NaN,NaN,Yes,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,69.0


In [6]:
# drop these columns because their values are all 0
df = df.drop(columns = ['socialPercentile', 'environmentPercentile', 'governancePercentile'])

In [7]:
# check the number of rows and columns in the dataframe
df.shape

(2000, 40)

In [8]:
# check the type of all columns
df.dtypes

Name                                          object
Industry                                      object
Headquarters                                  object
Region                                        object
Food and Agriculture Transformation           object
Decarbonisation and Energy Transformation     object
Nature and Biodiversity Transformation        object
Digital Transformation                        object
Financial Transformation                      object
Urban Transformation                          object
Social Transformation                         object
Profit_Margins                               float64
Market_Cap                                   float64
Total_Revenue                                float64
Enterprise_to_Revenue                        float64
Price_To_Sales_Trailing12Months              float64
Beta                                         float64
palmOil                                      float64
controversialWeapons                         f

#### 1. get dummies for categorical variables

In [9]:
# specify categorical variables and convert them to dummies
# drop the first column to avoid multicollinearity
df1 = pd.get_dummies(df, columns = ['Industry', 'Headquarters', 'Region', 'Food and Agriculture Transformation', 
                                    'Decarbonisation and Energy Transformation',
                                   'Nature and Biodiversity Transformation', 'Digital Transformation', 
                                    'Financial Transformation', 'Urban Transformation', 
                                    'Social Transformation', 'palmOil', 'controversialWeapons',
                                   'gambling', 'nuclear', 'furLeather', 'alcoholic', 'gmo',
                                   'catholic', 'animalTesting', 'tobacco', 'esgPerformance',
                                   'coal', 'pesticides', 'adult', 'smallArms', 'militaryContract'], 
                     drop_first = True)
df1.head()

,Name,Profit_Margins,Market_Cap,Total_Revenue,Enterprise_to_Revenue,Price_To_Sales_Trailing12Months,Beta,socialScore,peerCount,governanceScore,...,gmo_1.0,catholic_1.0,animalTesting_1.0,esgPerformance_LAG_PERF,esgPerformance_LEAD_PERF,esgPerformance_OUT_PERF,esgPerformance_UNDER_PERF,coal_1.0,pesticides_1.0,militaryContract_1.0
0,3M,0.15838,7.335740e+10,3.533300e+10,2.501,2.076172,0.945083,13.56,36.0,7.72,...,0,0,1,0,0,1,0,0,0,0
1,A.P. Moller - Maersk,0.32082,3.366736e+11,6.864000e+10,4.798,4.904919,1.420220,6.40,46.0,6.45,...,0,0,0,0,0,0,0,0,0,0
2,ABB,0.16023,5.122003e+10,2.900900e+10,1.904,1.765660,1.049832,6.65,19.0,6.81,...,0,0,0,0,0,0,1,0,0,0
3,AbbVie,0.21999,2.643067e+11,5.672500e+10,5.951,4.659439,0.748829,16.92,83.0,9.98,...,0,1,1,0,0,0,0,0,0,0
4,Abercrombie & Fitch,0.05469,8.505348e+08,3.744125e+09,0.413,0.227165,1.485312,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0


#### 2. Imputing missing data

In [10]:
# use KNNImputer model to impute missing values in the dataset
# use n_nerighbors = 3 for now and will optimize later in this file
df2 = df1.drop(columns = ['Name'])

# set up the imputer
imputer = KNNImputer(n_neighbors = 3)
imputed = imputer.fit_transform(df2)

# convert imputed dataset into a dataframe
df2_imputed = pd.DataFrame(imputed, columns = df2.columns)

In [11]:
# Before imputation: original dataset has a lot of missing values
df2.isna().sum()

Profit_Margins                     685
Market_Cap                         691
Total_Revenue                      689
Enterprise_to_Revenue              692
Price_To_Sales_Trailing12Months    697
                                  ... 
esgPerformance_OUT_PERF              0
esgPerformance_UNDER_PERF            0
coal_1.0                             0
pesticides_1.0                       0
militaryContract_1.0                 0
Length: 154, dtype: int64

In [12]:
# After imputation: filling in missing values
df2_imputed.isna().sum()

Profit_Margins                     0
Market_Cap                         0
Total_Revenue                      0
Enterprise_to_Revenue              0
Price_To_Sales_Trailing12Months    0
                                  ..
esgPerformance_OUT_PERF            0
esgPerformance_UNDER_PERF          0
coal_1.0                           0
pesticides_1.0                     0
militaryContract_1.0               0
Length: 154, dtype: int64

#### 3. Delete rows that have 0 ESG_rating

In [13]:
# 2000 rows and 154 columns after one-hot encoding
df2_imputed.shape

(2000, 154)

In [14]:
# check if there are null values for ESG_rating
df2_imputed['ESG_rating'].isnull().sum()

0

In [15]:
# dataset has 600 companies where ESG rating is equal to 0 (becuase of missing Refinitiv, MSCI, or Yahoo data)
df2_imputed[df['ESG_rating'] == 0]['ESG_rating'].count()

600

In [16]:
# removing the 600 companies becuase they are not helpful in training the models
df3 = df2_imputed[df2_imputed['ESG_rating'] != 0]
df3.shape

(1400, 154)

#### 4. Train-test split

In [17]:
# seperate independent and dependent variables
y = df3["ESG_rating"] 
x = df3.drop("ESG_rating",axis=1)

In [18]:
# split the data into 80% training and 20% testing
x_train, x_test = np.split(x, [int(.80 *len(x))])
y_train, y_test = np.split(y, [int(.80 *len(y))])

#### 5. Standardizing numerical variables

In [19]:
# all numerical variables
num_cols = ['Profit_Margins', 'Market_Cap', 'Total_Revenue','Enterprise_to_Revenue',
            'Price_To_Sales_Trailing12Months', 'Beta', 'socialScore', 'peerCount',
            'governanceScore', 'highestControversy','environmentScore']

x_train_trs = x_train.copy(deep=True)
x_test_trs = x_test.copy(deep=True)

# create scaler
scaler = StandardScaler()

# fit scaler on data
scaler.fit(x_train[num_cols])

# apply transform
x_train_trs[num_cols] = scaler.transform(x_train[num_cols])
x_test_trs[num_cols] = scaler.transform(x_test[num_cols])

# check transformed data
x_train_trs.head()

,Profit_Margins,Market_Cap,Total_Revenue,Enterprise_to_Revenue,Price_To_Sales_Trailing12Months,Beta,socialScore,peerCount,governanceScore,highestControversy,...,gmo_1.0,catholic_1.0,animalTesting_1.0,esgPerformance_LAG_PERF,esgPerformance_LEAD_PERF,esgPerformance_OUT_PERF,esgPerformance_UNDER_PERF,coal_1.0,pesticides_1.0,militaryContract_1.0
0,0.147334,-0.081842,-0.146270,-0.089470,-0.067961,-0.145842,1.186573,-0.880511,0.135263,1.171346,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.961226,-0.074584,-0.143450,0.025432,-0.054816,0.801311,-1.037508,-0.617597,-0.465629,1.171346,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.156603,-0.082453,-0.146805,-0.119333,-0.069404,0.062968,-0.959852,-1.327465,-0.295298,-0.015901,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0.456026,-0.076579,-0.144459,0.083107,-0.055956,-0.537061,2.230276,0.355184,1.204566,1.171346,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-0.372196,-0.083841,-0.148945,-0.193916,-0.076554,0.931068,-0.324104,0.162381,-1.022361,-0.807399,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Machine Learning Prediction Models

#### 1. Regression (can use as a benchmark)

In [18]:
# train regression model
reg_model = sm.OLS(y_train, x_train_trs)
results = reg_model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:             ESG_rating   R-squared:                       0.353
Model:                            OLS   Adj. R-squared:                  0.272
Method:                 Least Squares   F-statistic:                     4.380
Date:                Thu, 28 Jul 2022   Prob (F-statistic):           6.39e-40
Time:                        09:40:44   Log-Likelihood:                -4735.0
No. Observations:                1120   AIC:                             9720.
Df Residuals:                     995   BIC:                         1.035e+04
Df Model:                         124                                         
Covariance Type:            nonrobust                                         
                                                     coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------

In [19]:
# output prediction accuracy on the test set using MSE
y_pred_reg = results.predict(x_test_trs)
print("Mean squared error: %.2f" % mean_squared_error(y_test, y_pred_reg))

Mean squared error: 361.77


#### 2. KNN

In [20]:
# train KNN model
# try with k = 40 (square root of 1600)
knn_model =  KNeighborsRegressor(n_neighbors=40)
knn_model.fit(x_train_trs, y_train)

KNeighborsRegressor(n_neighbors=40)

In [21]:
# output prediction accuracy on the test set using MSE
y_pred_knn = knn_model.predict(x_test_trs)
print("Mean squared error: %.2f" % mean_squared_error(y_test, y_pred_knn))

Mean squared error: 366.40


#### 3. Random Forest

In [22]:
# train Random Forest model using default parameters
rf = RandomForestRegressor(n_estimators = 100, random_state = 42)
rf.fit(x_train_trs, y_train)

RandomForestRegressor(random_state=42)

In [23]:
# output prediction accuracy on the test set using MSE
y_pred_rf = rf.predict(x_test_trs)
print("Mean squared error: %.2f" % mean_squared_error(y_test, y_pred_rf))

Mean squared error: 289.59


#### 4. XGboost

In [26]:
# train XGboost model using default parameters
xgb_m = xgb.XGBRegressor(n_estimators = 100)
xgb_m.fit(x_train_trs, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, ...)

In [27]:
# output prediction accuracy on the test set using MSE
y_pred_xgb = xgb_m.predict(x_test_trs)
print("Mean squared error: %.2f" % mean_squared_error(y_test, y_pred_xgb))

Mean squared error: 309.05


#### 5. Neural Network

In [24]:
# train Neural Network model

n_features = x_train.shape[1]

# define the keras model
model = Sequential()
model.add(Dense(20, input_dim=n_features, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(10, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(1, activation='linear'))

# compile the keras model
model.compile(loss='mse', optimizer='adam')

# fit the keras model on the dataset
model.fit(x_train_trs, y_train, epochs=150, batch_size=20, verbose=2)

2022-07-28 09:43:05.640157: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/150
56/56 - 0s - loss: 2753.7551 - 386ms/epoch - 7ms/step
Epoch 2/150
56/56 - 0s - loss: 1577.3046 - 43ms/epoch - 769us/step
Epoch 3/150
56/56 - 0s - loss: 1195.5192 - 40ms/epoch - 716us/step
Epoch 4/150
56/56 - 0s - loss: 1070.9459 - 41ms/epoch - 731us/step
Epoch 5/150
56/56 - 0s - loss: 903.8596 - 40ms/epoch - 711us/step
Epoch 6/150
56/56 - 0s - loss: 714.4990 - 40ms/epoch - 709us/step
Epoch 7/150
56/56 - 0s - loss: 553.6744 - 38ms/epoch - 676us/step
Epoch 8/150
56/56 - 0s - loss: 456.2482 - 41ms/epoch - 733us/step
Epoch 9/150
56/56 - 0s - loss: 412.6538 - 45ms/epoch - 811us/step
Epoch 10/150
56/56 - 0s - loss: 392.9923 - 41ms/epoch - 733us/step
Epoch 11/150
56/56 - 0s - loss: 387.2195 - 44ms/epoch - 784us/step
Epoch 12/150
56/56 - 0s - loss: 377.3463 - 39ms/epoch - 705us/step
Epoch 13/150
56/56 - 0s - loss: 365.6310 - 44ms/epoch - 777us/step
Epoch 14/150
56/56 - 0s - loss: 362.0568 - 44ms/epoch - 788us/step
Epoch 15/150
56/56 - 0s - loss: 354.2134 - 42ms/epoch - 754us/step
E

Epoch 124/150
56/56 - 0s - loss: 246.1127 - 39ms/epoch - 691us/step
Epoch 125/150
56/56 - 0s - loss: 244.5460 - 38ms/epoch - 685us/step
Epoch 126/150
56/56 - 0s - loss: 244.6079 - 37ms/epoch - 664us/step
Epoch 127/150
56/56 - 0s - loss: 244.7458 - 38ms/epoch - 671us/step
Epoch 128/150
56/56 - 0s - loss: 242.5794 - 36ms/epoch - 642us/step
Epoch 129/150
56/56 - 0s - loss: 242.3143 - 37ms/epoch - 658us/step
Epoch 130/150
56/56 - 0s - loss: 247.7897 - 36ms/epoch - 644us/step
Epoch 131/150
56/56 - 0s - loss: 244.8259 - 36ms/epoch - 645us/step
Epoch 132/150
56/56 - 0s - loss: 241.0661 - 36ms/epoch - 641us/step
Epoch 133/150
56/56 - 0s - loss: 248.3084 - 36ms/epoch - 641us/step
Epoch 134/150
56/56 - 0s - loss: 243.0612 - 38ms/epoch - 682us/step
Epoch 135/150
56/56 - 0s - loss: 243.2188 - 37ms/epoch - 658us/step
Epoch 136/150
56/56 - 0s - loss: 242.0647 - 37ms/epoch - 652us/step
Epoch 137/150
56/56 - 0s - loss: 239.6159 - 40ms/epoch - 710us/step
Epoch 138/150
56/56 - 0s - loss: 244.4721 - 37ms

In [25]:
# output prediction accuracy on the test set using MSE
y_pred_nn = model.predict(x_test_trs)
print('Mean squared error: %.2f' % mean_squared_error(y_test, y_pred_nn))

Mean squared error: 1126.50


## Optimize the Random Forest Model

In [26]:
# write a function to preprocess data easily by inputing a dataframe and a k value for KNNImputer
def preprocessingData(dataframe, k):
    # impute missing values
    imputer = KNNImputer(n_neighbors = k)
    imputed = imputer.fit_transform(dataframe)
    df_imputed = pd.DataFrame(imputed, columns=dataframe.columns)
    
    df_cleaned = df_imputed[df_imputed['ESG_rating'] != 0]
    
    # separate independent and dependent variables
    y = df_cleaned["ESG_rating"] 
    x = df_cleaned.drop("ESG_rating",axis=1)
    
    # train test split
    x_train, x_test = np.split(x, [int(.80 *len(x))])
    y_train, y_test = np.split(y, [int(.80 *len(y))])
    
    # standardize numerical variables 
    num_cols = ['Profit_Margins', 'Market_Cap', 'Total_Revenue','Enterprise_to_Revenue',
                'Price_To_Sales_Trailing12Months', 'Beta', 'socialScore', 'peerCount',
                'governanceScore', 'highestControversy','environmentScore']
    x_train_trs = x_train.copy(deep=True)
    x_test_trs = x_test.copy(deep=True)
    scaler = StandardScaler() # create scaler
    scaler.fit(x_train[num_cols]) # fit scaler on data
    x_train_trs[num_cols] = scaler.transform(x_train[num_cols]) # apply transform on train set 
    x_test_trs[num_cols] = scaler.transform(x_test[num_cols]) # apply transform on test set

    return x_train_trs, y_train, x_test_trs, y_test, df_imputed, scaler

In [28]:
# optimize k value for KNNImputer that gives the lowest error

errors = {} # create a dictionary to store k value and erros
for num in range(1, 50, 2): # try k in this range, can modify as needed
    x_train_trs, y_train, x_test_trs, y_test, df_imputed, scaler = preprocessingData(dataframe = df2, k = num)
    
    # fit random forest model
    rf = RandomForestRegressor(random_state = 42)
    rf.fit(x_train_trs, y_train)
    
    # prediction error
    y_pred_rf = rf.predict(x_test_trs)
    error = mean_squared_error(y_test, y_pred_rf)
    errors[num] = error

print('K that gives lowest error: ', min(errors, key=errors.get))

K that gives lowest error:  27


#### Train RF using best the k for imputting missing values

In [29]:
# use the above processing data function to get training and testing dataset using best k = 27
x_train_trs, y_train, x_test_trs, y_test, df_imputed, scaler = preprocessingData(dataframe = df2, k = 27)

In [30]:
# use GridSearchCV to try all combinations of parameters
rf = RandomForestRegressor(random_state=42)

# specify parameters' values to train 
param_grid = {
    'bootstrap': [True],
    'n_estimators': [300, 400, 500],
    'max_features': ['auto'],
    'max_depth': [20, 30, 40],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12]
}

CV_rf = GridSearchCV(estimator=rf, param_grid=param_grid, cv = 3)
CV_rf.fit(x_train_trs, y_train)

GridSearchCV(cv=3, estimator=RandomForestRegressor(random_state=42),
             param_grid={'bootstrap': [True], 'max_depth': [20, 30, 40],
                         'max_features': ['auto'],
                         'min_samples_leaf': [3, 4, 5],
                         'min_samples_split': [8, 10, 12],
                         'n_estimators': [300, 400, 500]})

In [31]:
# output the set of parameters that gives the lowest error
CV_rf.best_params_

{'bootstrap': True,
 'max_depth': 20,
 'max_features': 'auto',
 'min_samples_leaf': 4,
 'min_samples_split': 8,
 'n_estimators': 400}

In [32]:
# use the best set of parameters to train the model again
rf_best = RandomForestRegressor(random_state = 42, bootstrap = 'True', max_depth = 20, max_features = 'auto',
                            min_samples_leaf = 4, min_samples_split = 8, n_estimators = 400)

rf_best.fit(x_train_trs, y_train)

RandomForestRegressor(bootstrap='True', max_depth=20, min_samples_leaf=4,
                      min_samples_split=8, n_estimators=400, random_state=42)

In [33]:
# evaluate the best model using MSE of the test set
y_pred_rf = rf_best.predict(x_test_trs)
print("Mean squared error: %.2f" % mean_squared_error(y_test, y_pred_rf))

Mean squared error: 260.94


Feature importances are provided by the fitted attribute feature_importances_ and they are computed as the mean and standard deviation of accumulation of the impurity decrease within each tree.

In [34]:
# rank all features by their importance score in training the model
# feature_importance_1 assign a score to input features based on how useful they are at predicting a target variable.
feat_labels = x_train_trs.columns
importance = rf_best.feature_importances_
imp_result = np.argsort(rf_best.feature_importances_)[::-1]
for i in range(len(imp_result)):
    print("%2d. %-*s %f" % (i + 1, 30, feat_labels[imp_result[i]], importance[imp_result[i]]))

 1. Headquarters_China             0.121063
 2. Price_To_Sales_Trailing12Months 0.104664
 3. Total_Revenue                  0.087988
 4. peerCount                      0.086994
 5. environmentScore               0.075216
 6. percentile                     0.071608
 7. highestControversy             0.071545
 8. socialScore                    0.061422
 9. Enterprise_to_Revenue          0.056832
10. Profit_Margins                 0.053534
11. Beta                           0.050107
12. governanceScore                0.048465
13. Market_Cap                     0.048081
14. Region_Europe & Central Asia   0.016032
15. esgPerformance_LAG_PERF        0.004776
16. animalTesting_1.0              0.004224
17. Region_North America           0.003437
18. Industry_Automobiles & Components 0.003349
19. Region_Middle East & North Africa 0.002856
20. Headquarters_United States of America 0.002852
21. Industry_Metals & Mining       0.002048
22. Industry_Real Estate           0.001852
23. Industry_Asset

## Predict Korean and Vietnam companies

In [35]:
# check if after imputed dataset has the same number of rows as the original dataset
# different number of columns because df_imputed is after getting dummies
print(df_imputed.shape)
print(df.shape)

(2000, 154)
(2000, 40)


In [36]:
# get columns we need to analyze results (columns before get dummies)
df_imputed['Headquarters'] = df['Headquarters']
df_imputed['Name'] = df['Name']
df_imputed['Industry'] = df['Industry']

In [37]:
# find companies in Korea and Vietnam
df_KR_VN = df_imputed[df_imputed['Headquarters'].isin(['Republic of Korea','Vietnam'])]

In [38]:
# check the dataset
df_KR_VN.head()

,Profit_Margins,Market_Cap,Total_Revenue,Enterprise_to_Revenue,Price_To_Sales_Trailing12Months,Beta,socialScore,peerCount,governanceScore,highestControversy,...,esgPerformance_LAG_PERF,esgPerformance_LEAD_PERF,esgPerformance_OUT_PERF,esgPerformance_UNDER_PERF,coal_1.0,pesticides_1.0,militaryContract_1.0,Headquarters,Name,Industry
91,0.033800,3.008890e+12,5.326080e+12,1.142000,0.564935,1.037283,8.130000,28.000000,7.780000,2.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Republic of Korea,Amorepacific Group,Personal & Household Products
227,0.148571,7.826040e+11,8.618536e+11,3.540963,3.443103,0.793538,9.567037,57.777778,7.460000,2.333333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Vietnam,BECAMEX Infrastructure Development,Real Estate
248,0.206229,1.065705e+12,7.095647e+11,8.686630,4.952568,0.904452,9.651481,63.111111,7.650370,2.259259,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Vietnam,Binh Duong Water Environment,Utilities
437,0.004820,2.253774e+12,1.134365e+13,0.382000,0.198681,-0.013759,10.347037,45.814815,8.783704,2.444444,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Republic of Korea,CJ Logistics Corporation,Logistics
508,0.055800,2.202349e+12,8.685208e+12,0.286000,0.253575,1.271193,11.275556,52.925926,9.144444,2.481481,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Republic of Korea,Daewoo Engineering & Construction,Construction & Engineering


In [39]:
# drop non-needed columns for prediction
x_KR_VN = df_KR_VN.drop(columns = ['ESG_rating', 'Name', 'Headquarters', 'Industry'])

x_KR_VN_trs = x_KR_VN.copy(deep=True)

# standardize numercial variables from the above best model's same scaler 
x_KR_VN_trs[num_cols] = scaler.transform(x_KR_VN[num_cols])

In [40]:
# there are 57 Korean and Vietnamese companies 
x_KR_VN.shape

(57, 153)

In [41]:
# use the best model to predict ESG_rating for Korean and Vietnamese companies 
ESG_rating_KR_VN = rf_best.predict(x_KR_VN_trs)

In [50]:
# take needed columns and put them in a result dataframe to check prediction
result = pd.DataFrame()
result['Name'] = df_KR_VN['Name']
result['Headquarters'] = df_KR_VN['Headquarters']
result['Industry'] = df_KR_VN['Industry']
result['ESG_predict'] = ESG_rating_KR_VN
result['ESG_rating'] = df_KR_VN['ESG_rating']

In [51]:
# check prediction results
result

,Name,Headquarters,Industry,ESG_predict,ESG_rating
91,Amorepacific Group,Republic of Korea,Personal & Household Products,65.289982,64.00
227,BECAMEX Infrastructure Development,Vietnam,Real Estate,34.968822,35.00
248,Binh Duong Water Environment,Vietnam,Utilities,7.532012,7.00
437,CJ Logistics Corporation,Republic of Korea,Logistics,41.319785,7.00
508,Daewoo Engineering & Construction,Republic of Korea,Construction & Engineering,70.011091,70.00
509,Daewoo Shipbuilding & Marine Engineering,Republic of Korea,Capital Goods,62.108674,60.00
553,DL E&C,Republic of Korea,Construction & Engineering,58.608861,0.00
554,DL Holdings,Republic of Korea,Conglomerates,62.062262,0.00
565,Dongwon Enterprise,Republic of Korea,Food & Beverage,7.532012,0.00
566,Doosan,Republic of Korea,Construction & Engineering,55.643715,74.00


In [52]:
# calculate the squared error for each company 
result['squared_error'] = (result['ESG_rating'] - result['ESG_predict']) ** 2
result.head()

,Name,Headquarters,Industry,ESG_predict,ESG_rating,squared_error
91,Amorepacific Group,Republic of Korea,Personal & Household Products,65.289982,64.0,1.664053
227,BECAMEX Infrastructure Development,Vietnam,Real Estate,34.968822,35.0,0.000972
248,Binh Duong Water Environment,Vietnam,Utilities,7.532012,7.0,0.283037
437,CJ Logistics Corporation,Republic of Korea,Logistics,41.319785,7.0,1177.847609
508,Daewoo Engineering & Construction,Republic of Korea,Construction & Engineering,70.011091,70.0,0.000123


In [48]:
# group by industry and calculate mean MSE for each industry
result.groupby(by="Industry")['squared_error'].mean().sort_values(ascending=False)

Industry
Conglomerates                        4047.636356
Logistics                            2009.338402
Food & Beverage                       707.730479
Construction & Engineering            493.103680
Automobiles & Components              385.078244
Capital Goods                         362.000883
Banks                                 224.329852
Oil & Gas                             159.135079
Metals & Mining                       145.412185
Passenger Transport                   111.839392
Electronics                            93.685294
Telecommunications                     85.887222
Utilities                              60.662339
Pension Funds                          56.731209
Agricultural Products                  56.731209
Construction Materials & Supplies      56.731209
Waste management                       56.731209
Retail                                 40.939887
Tyres & Rubber                         33.202218
Real Estate                            28.366090
Insurance  

In [53]:
# group by headquarters and calculate mean MSE for each country
result.groupby(by="Headquarters")['squared_error'].mean().sort_values(ascending=False)

Headquarters
Republic of Korea    470.32478
Vietnam               44.15583
Name: squared_error, dtype: float64